# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
import re
from collections import Counter
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    word_counts = Counter(text)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
#     print("int_to_vocab")
#     print(int_to_vocab)
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuations_dict = {
        ".": "||Period||",
        ",": "||Comma||",
        "\"": "||Quotation_Mark||",
        ";": "||Semicolon||",
        "!": "||Exclamation_Mark||",
        "?": "||Question_Mark||",
        "(": "||Left_Parentheses||",
        ")": "||Right_Parentheses||",
        "-": "||Dash||",
        "\n": "||Return||"
    }
    return punctuations_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    feature, target = [], []
    for idx in range(0, len(words), sequence_length):
        batch = words[idx:idx+sequence_length]
#         print(batch)
        
        feature.append(batch)
        batch_y = batch[-1]+1
        target.append(batch_y)
#         print(feature)
#         for ii in range(len(batch)):
#             batch_x = batch[ii]
#             batch_y = batch_x+1
#             target.append(batch_y)
#             feature.extend([batch_x])
    feature_tensors = torch.FloatTensor(feature)
    target_tensors = torch.FloatTensor(target)
    
    # return a dataloader
#     print(feature_tensors)
    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(data, 
                                              batch_size=batch_size)
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0.,   1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.],
        [ 15.,  16.,  17.,  18.,  19.],
        [ 20.,  21.,  22.,  23.,  24.],
        [ 25.,  26.,  27.,  28.,  29.],
        [ 30.,  31.,  32.,  33.,  34.],
        [ 35.,  36.,  37.,  38.,  39.],
        [ 40.,  41.,  42.,  43.,  44.],
        [ 45.,  46.,  47.,  48.,  49.]])

torch.Size([10])
tensor([  5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [10]:
if train_on_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [11]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=dropout, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """

        batch_size = nn_input.size(0)
        nn_input = nn_input.view(nn_input.size(0), -1)
        embeds = self.embedding(nn_input.long())
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        
        # reshape to be batch_size first
        out = out.view(batch_size, -1, self.output_size)
        out = out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [12]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda()

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    hidden = tuple([each.data for each in hidden])

    # zero accumulated gradients
    rnn.zero_grad()

    # get the output from the model
    output, hidden = rnn(inp, hidden)

    # calculate the loss and perform backprop
    loss = criterion(output.squeeze(), target.long())
    loss.backward()
    loss= loss.item()
    optimizer.step()
    return loss, hidden
# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [14]:
# Data params
# Sequence Length
sequence_length = 16  # of words in a sequence
# Batch Size
batch_size = 64

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [15]:
# Training parameters

# Number of Epochs
num_epochs = 3000
# Learning Rate
learning_rate = 0.001



# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
print(vocab_size)
# Output size
output_size = vocab_size

# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 5*int(len(train_loader.dataset) / (embedding_dim + output_size))
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

21388


### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [16]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

        
    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 16
batch_size: 64
learning_rate: 0.001
embedding_dim: 300
n_layers: 2
Training for 3000 epoch(s)...
Epoch:    1/3000  Loss: 7.6355538091659545

Epoch:    2/3000  Loss: 5.872751377753905

Epoch:    3/3000  Loss: 5.481700065078472

Epoch:    4/3000  Loss: 5.317049304051186

Epoch:    5/3000  Loss: 5.214368463246886

Epoch:    6/3000  Loss: 5.136035258405928

Epoch:    7/3000  Loss: 5.074308214724132

Epoch:    8/3000  Loss: 5.017919189210053

Epoch:    9/3000  Loss: 4.969350996849594

Epoch:   10/3000  Loss: 4.921557827192935

Epoch:   11/3000  Loss: 4.872283209615679

Epoch:   12/3000  Loss: 4.833799463188881

Epoch:   13/3000  Loss: 4.794799154579434

Epoch:   14/3000  Loss: 4.755663197021397

Epoch:   15/3000  Loss: 4.725045164198059

Epoch:   16/3000  Loss: 4.682623392405111

Epoch:   17/3000  Loss: 4.658289793301395

Epoch:   18/3000  Loss: 4.627390000083827

Epoch:   19/3000  Loss: 4.602226859803315

Epoch:   20/3000  Loss: 4.580942588339717

Epoch:   21/3000  Loss

Epoch:  188/3000  Loss: 3.7079964677720887

Epoch:  189/3000  Loss: 3.7096273816959457

Epoch:  190/3000  Loss: 3.699774859009052

Epoch:  191/3000  Loss: 3.712533006164428

Epoch:  192/3000  Loss: 3.7019353545218467

Epoch:  193/3000  Loss: 3.7120041220387963

Epoch:  194/3000  Loss: 3.719202084875271

Epoch:  195/3000  Loss: 3.7130622907566284

Epoch:  196/3000  Loss: 3.699062436921725

Epoch:  197/3000  Loss: 3.6994633097051888

Epoch:  198/3000  Loss: 3.701547770768438

Epoch:  199/3000  Loss: 3.700199755133225

Epoch:  200/3000  Loss: 3.7100902304446794

Epoch:  201/3000  Loss: 3.7019210819809363

Epoch:  202/3000  Loss: 3.7027087364897526

Epoch:  203/3000  Loss: 3.6994166817648675

Epoch:  204/3000  Loss: 3.701427973507466

Epoch:  205/3000  Loss: 3.6947010047947635

Epoch:  206/3000  Loss: 3.7076718155743746

Epoch:  207/3000  Loss: 3.704323507203848

Epoch:  208/3000  Loss: 3.701274918907682

Epoch:  209/3000  Loss: 3.6845223200170647

Epoch:  210/3000  Loss: 3.694458422786743

Epoch:  377/3000  Loss: 3.5980642739721884

Epoch:  378/3000  Loss: 3.5968604424517134

Epoch:  379/3000  Loss: 3.6006741578488617

Epoch:  380/3000  Loss: 3.593673915457917

Epoch:  381/3000  Loss: 3.588627263954942

Epoch:  382/3000  Loss: 3.5876410560410825

Epoch:  383/3000  Loss: 3.605088555306436

Epoch:  384/3000  Loss: 3.632507322302642

Epoch:  385/3000  Loss: 3.5857353202237325

Epoch:  386/3000  Loss: 3.591164227878733

Epoch:  387/3000  Loss: 3.575618731031188

Epoch:  388/3000  Loss: 3.5972307631671088

Epoch:  389/3000  Loss: 3.591535374710398

Epoch:  390/3000  Loss: 3.5625956488257846

Epoch:  391/3000  Loss: 3.557676534017657

Epoch:  392/3000  Loss: 3.5718678801259545

Epoch:  393/3000  Loss: 3.5650048732210107

Epoch:  394/3000  Loss: 3.554128958628728

Epoch:  395/3000  Loss: 3.577748378026636

Epoch:  396/3000  Loss: 3.5766640859137446

Epoch:  397/3000  Loss: 3.5702704162466268

Epoch:  398/3000  Loss: 3.556004632627923

Epoch:  399/3000  Loss: 3.573356644294292



Epoch:  566/3000  Loss: 3.5693232243972584

Epoch:  567/3000  Loss: 3.538353200621227

Epoch:  568/3000  Loss: 3.5389695936721997

Epoch:  569/3000  Loss: 3.5424195575933095

Epoch:  570/3000  Loss: 3.543190291773712

Epoch:  571/3000  Loss: 3.5472687686762496

Epoch:  572/3000  Loss: 3.5400464209021165

Epoch:  573/3000  Loss: 3.539241070041153

Epoch:  574/3000  Loss: 3.5355482230093394

Epoch:  575/3000  Loss: 3.535912066184831

Epoch:  576/3000  Loss: 3.529727464975911

Epoch:  577/3000  Loss: 3.541838124206228

Epoch:  578/3000  Loss: 3.528705860251811

Epoch:  579/3000  Loss: 3.534820934661357

Epoch:  580/3000  Loss: 3.5359268744147876

Epoch:  581/3000  Loss: 3.5505064754617472

Epoch:  582/3000  Loss: 3.5418793550178185

Epoch:  583/3000  Loss: 3.529117230295998

Epoch:  584/3000  Loss: 3.5351531858696044

Epoch:  585/3000  Loss: 3.535193259625703

Epoch:  586/3000  Loss: 3.5272074498484245

Epoch:  587/3000  Loss: 3.525564117354996

Epoch:  588/3000  Loss: 3.532480588599817



Epoch:  755/3000  Loss: 3.469137951098682

Epoch:  756/3000  Loss: 3.486355958932029

Epoch:  757/3000  Loss: 3.5236857293257073

Epoch:  758/3000  Loss: 3.4955399928766333

Epoch:  759/3000  Loss: 3.4894427128144714

Epoch:  760/3000  Loss: 3.4887886071998957

Epoch:  761/3000  Loss: 3.4939807813559005

Epoch:  762/3000  Loss: 3.480536925395786

Epoch:  763/3000  Loss: 3.470374799075822

Epoch:  764/3000  Loss: 3.502625894327525

Epoch:  765/3000  Loss: 3.4780553120833178

Epoch:  766/3000  Loss: 3.5021666886475273

Epoch:  767/3000  Loss: 3.4894681171215773

Epoch:  768/3000  Loss: 3.5090887573912277

Epoch:  769/3000  Loss: 3.506145604579238

Epoch:  770/3000  Loss: 3.5031145864458226

Epoch:  771/3000  Loss: 3.517779954402236

Epoch:  772/3000  Loss: 3.503377311675885

Epoch:  773/3000  Loss: 3.484313772138033

Epoch:  774/3000  Loss: 3.501450935817066

Epoch:  775/3000  Loss: 3.4898040275486135

Epoch:  776/3000  Loss: 3.494713022295561

Epoch:  777/3000  Loss: 3.5089737687401055


KeyboardInterrupt: 

In [ ]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

        
    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 16
batch_size: 64
learning_rate: 0.0001
embedding_dim: 300
n_layers: 2
Training for 3000 epoch(s)...
Epoch:    1/3000  Loss: 9.927245641708375

Epoch:    2/3000  Loss: 9.02769772193462

Epoch:    3/3000  Loss: 7.782626923408903

Epoch:    4/3000  Loss: 7.083254235483338

Epoch:    5/3000  Loss: 6.671719106001914

Epoch:    6/3000  Loss: 6.416002596013171

Epoch:    7/3000  Loss: 6.263115395634648

Epoch:    8/3000  Loss: 6.173454600004597

Epoch:    9/3000  Loss: 6.079519289916747

Epoch:   10/3000  Loss: 5.9827812755422665

Epoch:   11/3000  Loss: 5.893583059584643

Epoch:   12/3000  Loss: 5.82331772430893

Epoch:   13/3000  Loss: 5.760793973876743

Epoch:   14/3000  Loss: 5.706758129062937

Epoch:   15/3000  Loss: 5.658798049703394

Epoch:   16/3000  Loss: 5.611003027717774

Epoch:   17/3000  Loss: 5.577107125116955

Epoch:   18/3000  Loss: 5.5484686160607675

Epoch:   19/3000  Loss: 5.510022770523614

Epoch:   20/3000  Loss: 5.48522162410066

Epoch:   21/3000  Loss:

Epoch:  189/3000  Loss: 4.583297155754711

Epoch:  190/3000  Loss: 4.582935028864513

Epoch:  191/3000  Loss: 4.583048615198321

Epoch:  192/3000  Loss: 4.575864780647199

Epoch:  193/3000  Loss: 4.580725494673824

Epoch:  194/3000  Loss: 4.572587682240045

Epoch:  195/3000  Loss: 4.574253011926855

Epoch:  196/3000  Loss: 4.573353132889547

Epoch:  197/3000  Loss: 4.568586663729014

Epoch:  198/3000  Loss: 4.569317526713305

Epoch:  199/3000  Loss: 4.566060872904606

Epoch:  200/3000  Loss: 4.564940980052839

Epoch:  201/3000  Loss: 4.563058209063402

Epoch:  202/3000  Loss: 4.559796717903233

Epoch:  203/3000  Loss: 4.561212622339225

Epoch:  204/3000  Loss: 4.556698462993214

Epoch:  205/3000  Loss: 4.557166677391762

Epoch:  206/3000  Loss: 4.551921830795936

Epoch:  207/3000  Loss: 4.554923338402837

Epoch:  208/3000  Loss: 4.5511103201952645

Epoch:  209/3000  Loss: 4.552821037826801

Epoch:  210/3000  Loss: 4.547718435149516

Epoch:  211/3000  Loss: 4.5440322964117525

Epoch:  2

Epoch:  380/3000  Loss: 4.267072633541821

Epoch:  381/3000  Loss: 4.270066064479841

Epoch:  382/3000  Loss: 4.263192937239987

Epoch:  383/3000  Loss: 4.264794107693345

Epoch:  384/3000  Loss: 4.264885130213268

Epoch:  385/3000  Loss: 4.257181235487508

Epoch:  386/3000  Loss: 4.258735695060441

Epoch:  387/3000  Loss: 4.259306808016193

Epoch:  388/3000  Loss: 4.2597551526486805

Epoch:  389/3000  Loss: 4.2637927652637115

Epoch:  390/3000  Loss: 4.251759724283054

Epoch:  391/3000  Loss: 4.25843806545989

Epoch:  392/3000  Loss: 4.260021841758702

Epoch:  393/3000  Loss: 4.251980569688379

Epoch:  394/3000  Loss: 4.259610768984982

Epoch:  395/3000  Loss: 4.250876764205512

Epoch:  396/3000  Loss: 4.248954403961567

Epoch:  397/3000  Loss: 4.250533355774206

Epoch:  398/3000  Loss: 4.24687059629114

Epoch:  399/3000  Loss: 4.251058049919137

Epoch:  400/3000  Loss: 4.2462674725616845

Epoch:  401/3000  Loss: 4.252560740907759

Epoch:  402/3000  Loss: 4.248551780676595

Epoch:  40

Epoch:  577/3000  Loss: 4.132792152849322

Epoch:  578/3000  Loss: 4.1267339737078625

Epoch:  579/3000  Loss: 4.128068499942872

Epoch:  580/3000  Loss: 4.1181467403363685

Epoch:  581/3000  Loss: 4.12421193133813

Epoch:  582/3000  Loss: 4.119118914401627

Epoch:  583/3000  Loss: 4.125452821755656

Epoch:  584/3000  Loss: 4.126220307859296

Epoch:  585/3000  Loss: 4.124533335459081

Epoch:  586/3000  Loss: 4.114441144069492

Epoch:  587/3000  Loss: 4.116649704056137

Epoch:  588/3000  Loss: 4.118761983175092

Epoch:  589/3000  Loss: 4.121058555749746

Epoch:  590/3000  Loss: 4.118536063368367

Epoch:  591/3000  Loss: 4.1217215964495795

Epoch:  592/3000  Loss: 4.121323701024193

Epoch:  593/3000  Loss: 4.113071121112898

Epoch:  594/3000  Loss: 4.122647988673056

Epoch:  595/3000  Loss: 4.111166757228864

Epoch:  596/3000  Loss: 4.113297980907453

Epoch:  597/3000  Loss: 4.1104431913312345

Epoch:  598/3000  Loss: 4.115510333145527

Epoch:  599/3000  Loss: 4.108876552428498

Epoch:  

Epoch:  768/3000  Loss: 4.0295636905731484

Epoch:  769/3000  Loss: 4.020561505130457

Epoch:  770/3000  Loss: 4.025555344043999

Epoch:  771/3000  Loss: 4.025170776792016

Epoch:  772/3000  Loss: 4.025985802082189

Epoch:  773/3000  Loss: 4.026047954603123

Epoch:  774/3000  Loss: 4.033113645768193

Epoch:  775/3000  Loss: 4.034214116675709

Epoch:  776/3000  Loss: 4.0227777071032

Epoch:  777/3000  Loss: 4.0269943050073564

Epoch:  778/3000  Loss: 4.031039996754836

Epoch:  779/3000  Loss: 4.022760562590151

Epoch:  780/3000  Loss: 4.024261249329822

Epoch:  781/3000  Loss: 4.026481894756978

Epoch:  782/3000  Loss: 4.024986896394451

Epoch:  783/3000  Loss: 4.021101821030048

Epoch:  784/3000  Loss: 4.030090617531339

Epoch:  785/3000  Loss: 4.018057981396926

Epoch:  786/3000  Loss: 4.030218233607804

Epoch:  787/3000  Loss: 4.021580189979719

Epoch:  788/3000  Loss: 4.024042364102964

Epoch:  789/3000  Loss: 4.016070258329985

Epoch:  790/3000  Loss: 4.014692414094331

Epoch:  791

Epoch:  958/3000  Loss: 3.9760615683313625

Epoch:  959/3000  Loss: 3.9699567089672065

Epoch:  960/3000  Loss: 3.9730645219713074

Epoch:  961/3000  Loss: 3.9738267286499935

Epoch:  962/3000  Loss: 3.976181634942371

Epoch:  963/3000  Loss: 3.971691428041075

Epoch:  964/3000  Loss: 3.96122138973766

Epoch:  965/3000  Loss: 3.972839617701814

Epoch:  966/3000  Loss: 3.964089647910625

Epoch:  967/3000  Loss: 3.967149721631774

Epoch:  968/3000  Loss: 3.965285058957701

Epoch:  969/3000  Loss: 3.9718321409893362

Epoch:  970/3000  Loss: 3.9735856453121463

Epoch:  971/3000  Loss: 3.966566255254121

Epoch:  972/3000  Loss: 3.9713402495181658

Epoch:  973/3000  Loss: 3.964504565218149

Epoch:  974/3000  Loss: 3.9717076856698563

Epoch:  975/3000  Loss: 3.969496739307583

Epoch:  976/3000  Loss: 3.9652968252293688

Epoch:  977/3000  Loss: 3.9664661005635486

Epoch:  978/3000  Loss: 3.9582144869181373

Epoch:  979/3000  Loss: 3.967699711686845

Epoch:  980/3000  Loss: 3.961579421905097

E

Epoch: 1147/3000  Loss: 3.934317624117285

Epoch: 1148/3000  Loss: 3.938762259948678

Epoch: 1149/3000  Loss: 3.927720081381902

Epoch: 1150/3000  Loss: 3.9351173421682364

Epoch: 1151/3000  Loss: 3.9280950757583986

Epoch: 1152/3000  Loss: 3.9340059480765657

Epoch: 1153/3000  Loss: 3.9281100947602337

Epoch: 1154/3000  Loss: 3.921894593299051

Epoch: 1155/3000  Loss: 3.932601385631189

Epoch: 1156/3000  Loss: 3.9364475853544425

Epoch: 1157/3000  Loss: 3.92491243764262

Epoch: 1158/3000  Loss: 3.9294319434773635

Epoch: 1159/3000  Loss: 3.93413939842811

Epoch: 1160/3000  Loss: 3.91767672583649

Epoch: 1161/3000  Loss: 3.9338063596994814

Epoch: 1162/3000  Loss: 3.9270050136969092

Epoch: 1163/3000  Loss: 3.9244644288764894

Epoch: 1164/3000  Loss: 3.92994583734278

Epoch: 1165/3000  Loss: 3.9247164583917873

Epoch: 1166/3000  Loss: 3.925047924543221

Epoch: 1167/3000  Loss: 3.9217932686876895

Epoch: 1168/3000  Loss: 3.9270835713869396

Epoch: 1169/3000  Loss: 3.9242693288454915

Ep

In [ ]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 8
batch_size: 256
learning_rate: 0.0001
embedding_dim: 1000
n_layers: 3
Training for 3000 epoch(s)...
Epoch:    1/3000  Loss: 9.888944155375162

Epoch:    2/3000  Loss: 8.481809662128317

Epoch:    3/3000  Loss: 7.270790422373804

Epoch:    4/3000  Loss: 6.663805714968977

Epoch:    5/3000  Loss: 6.331771177533029

Epoch:    6/3000  Loss: 6.158400963092673

Epoch:    7/3000  Loss: 6.059639822203538

Epoch:    8/3000  Loss: 6.00237037724462

Epoch:    9/3000  Loss: 5.962919726316956

Epoch:   10/3000  Loss: 5.929200765456276

Epoch:   11/3000  Loss: 5.91067595317446

Epoch:   12/3000  Loss: 5.88859114044014

Epoch:   13/3000  Loss: 5.818020778962936

Epoch:   14/3000  Loss: 5.713282588432575

Epoch:   15/3000  Loss: 5.624934824581804

Epoch:   16/3000  Loss: 5.5624129065151875

Epoch:   17/3000  Loss: 5.516269770435903

Epoch:   18/3000  Loss: 5.477379350552614

Epoch:   19/3000  Loss: 5.445544637482742

Epoch:   20/3000  Loss: 5.418590889853992

Epoch:   21/3000  Loss:

Epoch:  190/3000  Loss: 4.231658464190604

Epoch:  191/3000  Loss: 4.227922789803866

Epoch:  192/3000  Loss: 4.227247528098095

Epoch:  193/3000  Loss: 4.221027190109779

Epoch:  194/3000  Loss: 4.221546050323837

Epoch:  195/3000  Loss: 4.218442141872713

Epoch:  196/3000  Loss: 4.2169736264765945

Epoch:  197/3000  Loss: 4.212291082294508

Epoch:  198/3000  Loss: 4.211051874599238

Epoch:  199/3000  Loss: 4.206424755337594

Epoch:  200/3000  Loss: 4.207415430573211

Epoch:  201/3000  Loss: 4.2035459129289645

Epoch:  202/3000  Loss: 4.201158343238392

Epoch:  203/3000  Loss: 4.2028868773887895

Epoch:  204/3000  Loss: 4.1963885647126995

Epoch:  205/3000  Loss: 4.1971663491479285

Epoch:  206/3000  Loss: 4.192559139207862

Epoch:  207/3000  Loss: 4.193521774226221

Epoch:  208/3000  Loss: 4.191504150149466

Epoch:  209/3000  Loss: 4.182039977764261

Epoch:  210/3000  Loss: 4.186182845872024

Epoch:  211/3000  Loss: 4.180613050789669

Epoch:  212/3000  Loss: 4.178598889537241

Epoch:

Epoch:  380/3000  Loss: 3.864411653321365

Epoch:  381/3000  Loss: 3.8608237359715605

Epoch:  382/3000  Loss: 3.861357044351512

Epoch:  383/3000  Loss: 3.8623777060673157

Epoch:  384/3000  Loss: 3.8608371975778164

Epoch:  385/3000  Loss: 3.8538774676706598

Epoch:  386/3000  Loss: 3.8486099643268803

Epoch:  387/3000  Loss: 3.851883052957469

Epoch:  388/3000  Loss: 3.8484718108999316

Epoch:  389/3000  Loss: 3.8452376798651686

Epoch:  390/3000  Loss: 3.845026122564557

Epoch:  391/3000  Loss: 3.84378574415185

Epoch:  392/3000  Loss: 3.843291636171012

Epoch:  393/3000  Loss: 3.836804767586719

Epoch:  394/3000  Loss: 3.839034704778386

Epoch:  395/3000  Loss: 3.8338858862032836

Epoch:  396/3000  Loss: 3.8379438186513966

Epoch:  397/3000  Loss: 3.8343907027408997

Epoch:  398/3000  Loss: 3.835340862712641

Epoch:  399/3000  Loss: 3.829510726051769

Epoch:  400/3000  Loss: 3.8269414452300676

Epoch:  401/3000  Loss: 3.825769441429226

Epoch:  402/3000  Loss: 3.8241010737145085



Epoch:  569/3000  Loss: 3.649572409706554

Epoch:  570/3000  Loss: 3.6460549924565457

Epoch:  571/3000  Loss: 3.6455925119334256

Epoch:  572/3000  Loss: 3.643196384934173

Epoch:  573/3000  Loss: 3.6433269621311934

Epoch:  574/3000  Loss: 3.644326003940626

Epoch:  575/3000  Loss: 3.641958957979049

Epoch:  576/3000  Loss: 3.645352477588873

Epoch:  577/3000  Loss: 3.645396911138776

Epoch:  578/3000  Loss: 3.645276566757553

Epoch:  579/3000  Loss: 3.640157420607819

Epoch:  580/3000  Loss: 3.6361152100837093

Epoch:  581/3000  Loss: 3.6390769673489975

Epoch:  582/3000  Loss: 3.6488452259151414

Epoch:  583/3000  Loss: 3.6375268393549427

Epoch:  584/3000  Loss: 3.635580441595494

Epoch:  585/3000  Loss: 3.639664686685321

Epoch:  586/3000  Loss: 3.6399446602525383

Epoch:  587/3000  Loss: 3.6318660390788113

Epoch:  588/3000  Loss: 3.6306359181458925

Epoch:  589/3000  Loss: 3.6381869469565906

Epoch:  590/3000  Loss: 3.6349846313739644

Epoch:  591/3000  Loss: 3.627906213409599


In [53]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 8
batch_size: 256
learning_rate: 0.0001
embedding_dim: 1000
n_layers: 2
Training for 3000 epoch(s)...
Epoch:    1/3000  Loss: 9.824233853022257

Epoch:    2/3000  Loss: 8.382291491278286

Epoch:    3/3000  Loss: 7.169598748218054

Epoch:    4/3000  Loss: 6.598437259937155

Epoch:    5/3000  Loss: 6.2991787132175485

Epoch:    6/3000  Loss: 6.134664025799982

Epoch:    7/3000  Loss: 6.029405585102651

Epoch:    8/3000  Loss: 5.935897772339569

Epoch:    9/3000  Loss: 5.8358773878251

Epoch:   10/3000  Loss: 5.7491682414350835

Epoch:   11/3000  Loss: 5.66413636043154

Epoch:   12/3000  Loss: 5.602039262618142

Epoch:   13/3000  Loss: 5.545062911373446

Epoch:   14/3000  Loss: 5.499353295907207

Epoch:   15/3000  Loss: 5.456617101033529

Epoch:   16/3000  Loss: 5.422627278031974

Epoch:   17/3000  Loss: 5.390671954757866

Epoch:   18/3000  Loss: 5.358663776003081

Epoch:   19/3000  Loss: 5.329554732092496

Epoch:   20/3000  Loss: 5.303960079982363

Epoch:   21/3000  Loss

Epoch:  188/3000  Loss: 3.6730805528574977

Epoch:  189/3000  Loss: 3.6716800766429682

Epoch:  190/3000  Loss: 3.6643114703825153

Epoch:  191/3000  Loss: 3.6670298242020882

Epoch:  192/3000  Loss: 3.661429737354147

Epoch:  193/3000  Loss: 3.6599779627789024

Epoch:  194/3000  Loss: 3.6516244099057955

Epoch:  195/3000  Loss: 3.6541932933631984

Epoch:  196/3000  Loss: 3.6494464918114673

Epoch:  197/3000  Loss: 3.644149468410974

Epoch:  198/3000  Loss: 3.6410404868509576

Epoch:  199/3000  Loss: 3.6367009979554976

Epoch:  200/3000  Loss: 3.62786553700765

Epoch:  201/3000  Loss: 3.628086361391791

Epoch:  202/3000  Loss: 3.6229642736500707

Epoch:  203/3000  Loss: 3.6174962378096307

Epoch:  204/3000  Loss: 3.6197487792749516

Epoch:  205/3000  Loss: 3.607819215730689

Epoch:  206/3000  Loss: 3.6092986556305284

Epoch:  207/3000  Loss: 3.607667919137012

Epoch:  208/3000  Loss: 3.5965434721146505

Epoch:  209/3000  Loss: 3.5986521183759317

Epoch:  210/3000  Loss: 3.5974436716101

KeyboardInterrupt: 

In [50]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 8
batch_size: 256
learning_rate: 0.0001
embedding_dim: 300
n_layers: 2
Training for 3000 epoch(s)...
Epoch:    1/3000  Loss: 9.81199777285258

Epoch:    2/3000  Loss: 8.006715022558453

Epoch:    3/3000  Loss: 6.743958465532325

Epoch:    4/3000  Loss: 6.281857250476706

Epoch:    5/3000  Loss: 6.083624680836995

Epoch:    6/3000  Loss: 5.976886859981493

Epoch:    7/3000  Loss: 5.867835347405795

Epoch:    8/3000  Loss: 5.745765932675066

Epoch:    9/3000  Loss: 5.6365704141814135

Epoch:   10/3000  Loss: 5.5505092851046856

Epoch:   11/3000  Loss: 5.481542950114985

Epoch:   12/3000  Loss: 5.417606043541569

Epoch:   13/3000  Loss: 5.3647027487042305

Epoch:   14/3000  Loss: 5.316270245080707

Epoch:   15/3000  Loss: 5.270434730354396

Epoch:   16/3000  Loss: 5.228882394987961

Epoch:   17/3000  Loss: 5.18576642617412

Epoch:   18/3000  Loss: 5.147305793323736

Epoch:   19/3000  Loss: 5.112871199640734

Epoch:   20/3000  Loss: 5.076744826086636

Epoch:   21/3000  Los

Epoch:  188/3000  Loss: 3.0838823455503617

Epoch:  189/3000  Loss: 3.07588313256187

Epoch:  190/3000  Loss: 3.0715731686559216

Epoch:  191/3000  Loss: 3.0672682307232386

Epoch:  192/3000  Loss: 3.0645530980208826

Epoch:  193/3000  Loss: 3.063295350129577

Epoch:  194/3000  Loss: 3.0560990064993674

Epoch:  195/3000  Loss: 3.0508512749069037

Epoch:  196/3000  Loss: 3.0481489691241035

Epoch:  197/3000  Loss: 3.043376108147632

Epoch:  198/3000  Loss: 3.0449800946246617

Epoch:  199/3000  Loss: 3.043288561393475

Epoch:  200/3000  Loss: 3.0374663901054997

Epoch:  201/3000  Loss: 3.031935098801536

Epoch:  202/3000  Loss: 3.032428492622814

Epoch:  203/3000  Loss: 3.0291035252055902

Epoch:  204/3000  Loss: 3.02245636041137

Epoch:  205/3000  Loss: 3.0199029982775105

Epoch:  206/3000  Loss: 3.0198470652788534

Epoch:  207/3000  Loss: 3.0140105548946337

Epoch:  208/3000  Loss: 3.01569270868411

Epoch:  209/3000  Loss: 3.0086400366377557

Epoch:  210/3000  Loss: 3.0017098761152945


KeyboardInterrupt: 

In [45]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 8
batch_size: 256
learning_rate: 0.0006
embedding_dim: 300
n_layers: 2
Training for 3000 epoch(s)...
Epoch:    1/3000  Loss: 8.081838583946228

Epoch:    1/3000  Loss: 6.120721523761749

Epoch:    2/3000  Loss: 5.652303864093537

Epoch:    2/3000  Loss: 5.4752587890625

Epoch:    3/3000  Loss: 5.304156993297821

Epoch:    3/3000  Loss: 5.247151322364807

Epoch:    4/3000  Loss: 5.141844925981887

Epoch:    4/3000  Loss: 5.094993231296539

Epoch:    5/3000  Loss: 4.996995662121063

Epoch:    5/3000  Loss: 4.954444153308868

Epoch:    6/3000  Loss: 4.8597314469357755

Epoch:    6/3000  Loss: 4.810218162536621

Epoch:    7/3000  Loss: 4.706127327046496

Epoch:    7/3000  Loss: 4.657008006572723

Epoch:    8/3000  Loss: 4.562745825787808

Epoch:    8/3000  Loss: 4.528742846250534

Epoch:    9/3000  Loss: 4.435271200220636

Epoch:    9/3000  Loss: 4.402329622507096

Epoch:   10/3000  Loss: 4.323272229255514

Epoch:   10/3000  Loss: 4.2924572014808655

Epoch:   11/3000  Loss

KeyboardInterrupt: 

In [40]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 8
batch_size: 256
learning_rate: 0.001
embedding_dim: 1000
n_layers: 2
Training for 1000 epoch(s)...
Epoch:    1/1000  Loss: 7.634065723419189

Epoch:    1/1000  Loss: 5.955679357051849

Epoch:    2/1000  Loss: 5.511626470849869

Epoch:    2/1000  Loss: 5.392672443389893

Epoch:    3/1000  Loss: 5.242239536123073

Epoch:    3/1000  Loss: 5.202348861694336

Epoch:    4/1000  Loss: 5.090768749155897

Epoch:    4/1000  Loss: 5.053709847927093

Epoch:    5/1000  Loss: 4.957887150378937

Epoch:    5/1000  Loss: 4.916481184959411

Epoch:    6/1000  Loss: 4.816588998348155

Epoch:    6/1000  Loss: 4.771510138511657

Epoch:    7/1000  Loss: 4.675779872244977

Epoch:    7/1000  Loss: 4.61408976316452

Epoch:    8/1000  Loss: 4.525214684263188

Epoch:    8/1000  Loss: 4.4839259624481205

Epoch:    9/1000  Loss: 4.4155620574951175

Epoch:    9/1000  Loss: 4.385436755418778

Epoch:   10/1000  Loss: 4.3220946910533495

Epoch:   10/1000  Loss: 4.297259469032287

Epoch:   11/1000  Lo

KeyboardInterrupt: 

In [37]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 8
batch_size: 256
learning_rate: 0.001
embedding_dim: 300
n_layers: 2
Training for 1000 epoch(s)...
Epoch:    1/1000  Loss: 7.404851336479187

Epoch:    1/1000  Loss: 5.821729054450989

Epoch:    2/1000  Loss: 5.358581626161616

Epoch:    2/1000  Loss: 5.223323135375977

Epoch:    3/1000  Loss: 5.0404612561489675

Epoch:    3/1000  Loss: 4.9523903131484985

Epoch:    4/1000  Loss: 4.795686090753434

Epoch:    4/1000  Loss: 4.70781599521637

Epoch:    5/1000  Loss: 4.5727390157415515

Epoch:    5/1000  Loss: 4.492256083488464

Epoch:    6/1000  Loss: 4.378561362814396

Epoch:    6/1000  Loss: 4.320309098958969

Epoch:    7/1000  Loss: 4.2158536048645665

Epoch:    7/1000  Loss: 4.169388976097107

Epoch:    8/1000  Loss: 4.071609021247702

Epoch:    8/1000  Loss: 4.03673201918602

Epoch:    9/1000  Loss: 3.9398714704716458

Epoch:    9/1000  Loss: 3.8923615515232086

Epoch:   10/1000  Loss: 3.7964081541020818

Epoch:   10/1000  Loss: 3.7774944508075716

Epoch:   11/1000 

KeyboardInterrupt: 

In [32]:
from workspace_utils import active_session

with active_session():
    """
    DON'T MODIFY ANYTHING IN THIS CELL

    """
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('/trained_rnn', trained_rnn)
    print('Model Trained and Saved')
    print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))
    

sequence_length: 8
batch_size: 256
learning_rate: 0.0001
embedding_dim: 300
n_layers: 2
Training for 1000 epoch(s)...
Epoch:    1/1000  Loss: 9.978208637237548

Epoch:    1/1000  Loss: 9.891534905433655

Epoch:    2/1000  Loss: 9.703820244809414

Epoch:    2/1000  Loss: 9.317116713523864

Epoch:    3/1000  Loss: 8.797745177086364

Epoch:    3/1000  Loss: 8.388129572868348

Epoch:    4/1000  Loss: 8.063018916515594

Epoch:    4/1000  Loss: 7.801344282627106

Epoch:    5/1000  Loss: 7.569390552601916

Epoch:    5/1000  Loss: 7.386419682502747

Epoch:    6/1000  Loss: 7.206695465331382

Epoch:    6/1000  Loss: 7.0723322796821595

Epoch:    7/1000  Loss: 6.9276548953766515

Epoch:    7/1000  Loss: 6.83409030675888

Epoch:    8/1000  Loss: 6.719126171761371

Epoch:    8/1000  Loss: 6.6382817673683165

Epoch:    9/1000  Loss: 6.5461004155747435

Epoch:    9/1000  Loss: 6.49627852678299

Epoch:   10/1000  Loss: 6.4200994166922065

Epoch:   10/1000  Loss: 6.382921674251556

Epoch:   11/1000  L

Epoch:   95/1000  Loss: 4.940312369326327

Epoch:   95/1000  Loss: 4.958992207050324

Epoch:   96/1000  Loss: 4.936118963931469

Epoch:   96/1000  Loss: 4.949335548877716

Epoch:   97/1000  Loss: 4.93604360336953

Epoch:   97/1000  Loss: 4.948787691593171

Epoch:   98/1000  Loss: 4.930171096071284

Epoch:   98/1000  Loss: 4.943124990463257

Epoch:   99/1000  Loss: 4.921475353646786

Epoch:   99/1000  Loss: 4.938651809692383

Epoch:  100/1000  Loss: 4.922233916343527

Epoch:  100/1000  Loss: 4.943068854808807

Epoch:  101/1000  Loss: 4.915639816446507

Epoch:  101/1000  Loss: 4.929651412963867

Epoch:  102/1000  Loss: 4.914856050369587

Epoch:  102/1000  Loss: 4.9260815906524655

Epoch:  103/1000  Loss: 4.909216619045177

Epoch:  103/1000  Loss: 4.921838624477386

Epoch:  104/1000  Loss: 4.905169551930529

Epoch:  104/1000  Loss: 4.926055064201355

Epoch:  105/1000  Loss: 4.9008836036032815

Epoch:  105/1000  Loss: 4.917802314758301

Epoch:  106/1000  Loss: 4.8956652377514125

Epoch:  1

Epoch:  190/1000  Loss: 4.664548952579498

Epoch:  191/1000  Loss: 4.648885820267048

Epoch:  191/1000  Loss: 4.658499553203582

Epoch:  192/1000  Loss: 4.644682230847947

Epoch:  192/1000  Loss: 4.657595139741898

Epoch:  193/1000  Loss: 4.641193174808583

Epoch:  193/1000  Loss: 4.654971945285797

Epoch:  194/1000  Loss: 4.638999159792636

Epoch:  194/1000  Loss: 4.654784479141235

Epoch:  195/1000  Loss: 4.641525100139861

Epoch:  195/1000  Loss: 4.653811945915222

Epoch:  196/1000  Loss: 4.639534002669314

Epoch:  196/1000  Loss: 4.653953876495361

Epoch:  197/1000  Loss: 4.633316624418218

Epoch:  197/1000  Loss: 4.652689502239228

Epoch:  198/1000  Loss: 4.632338895189001

Epoch:  198/1000  Loss: 4.651049538850784

Epoch:  199/1000  Loss: 4.63409705263503

Epoch:  199/1000  Loss: 4.648226745128632

Epoch:  200/1000  Loss: 4.63485803604126

Epoch:  200/1000  Loss: 4.646501636505127

Epoch:  201/1000  Loss: 4.63280399606583

Epoch:  201/1000  Loss: 4.646165707111359

Epoch:  202/10

Epoch:  286/1000  Loss: 4.534088288976791

Epoch:  286/1000  Loss: 4.550059700012207

Epoch:  287/1000  Loss: 4.531885511317151

Epoch:  287/1000  Loss: 4.548173687458038

Epoch:  288/1000  Loss: 4.537433839351573

Epoch:  288/1000  Loss: 4.549674181938172

Epoch:  289/1000  Loss: 4.533675679754704

Epoch:  289/1000  Loss: 4.550656433105469

Epoch:  290/1000  Loss: 4.530772259894838

Epoch:  290/1000  Loss: 4.53951824426651

Epoch:  291/1000  Loss: 4.534897568885317

Epoch:  291/1000  Loss: 4.542259608507156

Epoch:  292/1000  Loss: 4.5322213203349015

Epoch:  292/1000  Loss: 4.548176305294037

Epoch:  293/1000  Loss: 4.527793367872847

Epoch:  293/1000  Loss: 4.542443614006043

Epoch:  294/1000  Loss: 4.52877098144369

Epoch:  294/1000  Loss: 4.542702199220657

Epoch:  295/1000  Loss: 4.525372160241959

Epoch:  295/1000  Loss: 4.546953125

Epoch:  296/1000  Loss: 4.527573342018939

Epoch:  296/1000  Loss: 4.5424331700801845

Epoch:  297/1000  Loss: 4.524502005475632

Epoch:  297/1000 

Epoch:  381/1000  Loss: 4.479252301454544

Epoch:  382/1000  Loss: 4.46352194725199

Epoch:  382/1000  Loss: 4.489015257358551

Epoch:  383/1000  Loss: 4.46534346012359

Epoch:  383/1000  Loss: 4.487105865478515

Epoch:  384/1000  Loss: 4.461186350152848

Epoch:  384/1000  Loss: 4.480562608242035

Epoch:  385/1000  Loss: 4.466663469152247

Epoch:  385/1000  Loss: 4.4813993966579435

Epoch:  386/1000  Loss: 4.466244205515435

Epoch:  386/1000  Loss: 4.475033702850342

Epoch:  387/1000  Loss: 4.463886915369237

Epoch:  387/1000  Loss: 4.475060620307922

Epoch:  388/1000  Loss: 4.458168523869616

Epoch:  388/1000  Loss: 4.477310931682586

Epoch:  389/1000  Loss: 4.46323941920666

Epoch:  389/1000  Loss: 4.475747730731964

Epoch:  390/1000  Loss: 4.455548115994068

Epoch:  390/1000  Loss: 4.473899348974228

Epoch:  391/1000  Loss: 4.457909445052452

Epoch:  391/1000  Loss: 4.471949026584626

Epoch:  392/1000  Loss: 4.464323496311269

Epoch:  392/1000  Loss: 4.474196871519089

Epoch:  393/1

...

Epoch:  572/1000  Loss: 4.354943209886551

Epoch:  573/1000  Loss: 4.33417781464597

Epoch:  573/1000  Loss: 4.354273951053619

Epoch:  574/1000  Loss: 4.335605915556562

Epoch:  574/1000  Loss: 4.351063535213471

Epoch:  575/1000  Loss: 4.33615899288908

Epoch:  575/1000  Loss: 4.351077581644058

Epoch:  576/1000  Loss: 4.337125956758539

Epoch:  576/1000  Loss: 4.355869269371032

Epoch:  577/1000  Loss: 4.334921278852097

Epoch:  577/1000  Loss: 4.347891703844071

Epoch:  578/1000  Loss: 4.333561982499792

Epoch:  578/1000  Loss: 4.349157971143723

Epoch:  579/1000  Loss: 4.3282496685677385

Epoch:  579/1000  Loss: 4.346790586709976

Epoch:  580/1000  Loss: 4.336128584882046

Epoch:  580/1000  Loss: 4.347709386348725

Epoch:  581/1000  Loss: 4.334274266628509

Epoch:  581/1000  Loss: 4.347970839738846

Epoch:  582/1000  Loss: 4.330031889042956

Epoch:  582/1000  Loss: 4.348928171396255

Epoch:  583/1000  Loss: 4.32746283855844

Epoch:  583/1000  Loss: 4.350152423381806

Epoch:  584/1

Epoch:  668/1000  Loss: 4.283873777186617

Epoch:  668/1000  Loss: 4.3122657513618465

Epoch:  669/1000  Loss: 4.288270670302371

Epoch:  669/1000  Loss: 4.297441620826721

Epoch:  670/1000  Loss: 4.2819348761375915

Epoch:  670/1000  Loss: 4.311282968521118

Epoch:  671/1000  Loss: 4.291985841507607

Epoch:  671/1000  Loss: 4.299888900518417

Epoch:  672/1000  Loss: 4.2843761423800855

Epoch:  672/1000  Loss: 4.303737391233444

Epoch:  673/1000  Loss: 4.288900407831719

Epoch:  673/1000  Loss: 4.3080549919605255

Epoch:  674/1000  Loss: 4.294618340756031

Epoch:  674/1000  Loss: 4.303298454284668

Epoch:  675/1000  Loss: 4.282866334915161

Epoch:  675/1000  Loss: 4.302434825897217

Epoch:  676/1000  Loss: 4.277169502542374

Epoch:  676/1000  Loss: 4.301446244716645

Epoch:  677/1000  Loss: 4.283259111769656

Epoch:  677/1000  Loss: 4.305021790266037

Epoch:  678/1000  Loss: 4.283762416433781

Epoch:  678/1000  Loss: 4.30198407292366

Epoch:  679/1000  Loss: 4.286384624115964

Epoch:  

Epoch:  763/1000  Loss: 4.270302724838257

Epoch:  764/1000  Loss: 4.257269400738656

Epoch:  764/1000  Loss: 4.275276415348053

Epoch:  765/1000  Loss: 4.255350609028593

Epoch:  765/1000  Loss: 4.270115431547165

Epoch:  766/1000  Loss: 4.255314935521876

Epoch:  766/1000  Loss: 4.2736507999897

Epoch:  767/1000  Loss: 4.258672630025985

Epoch:  767/1000  Loss: 4.267810255289078

Epoch:  768/1000  Loss: 4.2519241261989515

Epoch:  768/1000  Loss: 4.271369577646255

Epoch:  769/1000  Loss: 4.2620699253488095

Epoch:  769/1000  Loss: 4.2699126327037815

Epoch:  770/1000  Loss: 4.253290997160242

Epoch:  770/1000  Loss: 4.2743379855155945

Epoch:  771/1000  Loss: 4.254784325335888

Epoch:  771/1000  Loss: 4.269423543214798

Epoch:  772/1000  Loss: 4.253190507280066

Epoch:  772/1000  Loss: 4.273460961580277

Epoch:  773/1000  Loss: 4.256890147797605

Epoch:  773/1000  Loss: 4.270639872550964

Epoch:  774/1000  Loss: 4.2481476286624344

Epoch:  774/1000  Loss: 4.263577696084976

Epoch:  

Epoch:  859/1000  Loss: 4.231598692751946

Epoch:  859/1000  Loss: 4.248168864250183

Epoch:  860/1000  Loss: 4.224963950096293

Epoch:  860/1000  Loss: 4.241870261430741

Epoch:  861/1000  Loss: 4.230077830781328

Epoch:  861/1000  Loss: 4.2481224310398105

Epoch:  862/1000  Loss: 4.227749957429602

Epoch:  862/1000  Loss: 4.2508560502529145

Epoch:  863/1000  Loss: 4.22686569639977

Epoch:  863/1000  Loss: 4.2389839029312135

Epoch:  864/1000  Loss: 4.235322358760428

Epoch:  864/1000  Loss: 4.2427764880657195

Epoch:  865/1000  Loss: 4.234678884262734

Epoch:  865/1000  Loss: 4.244241752624512

Epoch:  866/1000  Loss: 4.231195806949697

Epoch:  866/1000  Loss: 4.246909085512161

Epoch:  867/1000  Loss: 4.23670682704195

Epoch:  867/1000  Loss: 4.242840002775193

Epoch:  868/1000  Loss: 4.2300709359189295

Epoch:  868/1000  Loss: 4.238955169916153

Epoch:  869/1000  Loss: 4.228929347180306

Epoch:  869/1000  Loss: 4.24471519947052

Epoch:  870/1000  Loss: 4.223680608830554

Epoch:  8

KeyboardInterrupt: 

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [27]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 16
batch_size: 64
learning_rate: 0.001
embedding_dim: 300
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: george's pretty thought very if do don't with look one your at have of george: very come can't see that me no) in me of a a in of!? i,?,, i?? jerry: thought one your your- what that me of!,?,?? you i,,,?? jerry: turns her go do he he i'm) george: water think) george: alright too if do don't that of a in what of george: ready talk these her we),,,? jerry: thought back was-!,,?? you, you you, i,? you? you,?? i,,, i,?,,, i,, i you i, i,,?,, jerry: when hey)! you jerry:!,?,? you jerry:!? i jerry: jerry he well well i'm he my) george: alright one your do like all he)!, you i you, you? i,,, i?, jerry: alright very come one do- kramer: thought could hey)!, i, i,?, jerry: alright back are that me of a in of a a! you you jerry: of george: thought very he's he's see yeah- of a in no do don't that no know don't that of! jerry: kramer: what with are kramer: kramer: kramer: me of!,,,,??,?? i, jerry: very that's do don't with was well) in no do- no hey do don't that kramer: me no not- kramer:

In [25]:
"""
sequence_length: 16
batch_size: 256
learning_rate: 0.001
embedding_dim: 300
n_layers: 2
Training for 300 epoch(s)
"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 16
batch_size: 64
learning_rate: 0.001
embedding_dim: 300
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: project glasses george's george's pretty thought very come he's one do don't for have yeah he well) a a a!??, you,,, you, jerry: alright one-! i you,, you i i jerry: me of george: ready these one" no hey know on no hey) a! jerry: kramer: of george: ready talk these her have of george: few could do don't for have kramer: no know don't for have kramer: kramer: kramer: me what me of a!,,?? i jerry: for have that what that of!,,??, i,? you jerry: is he well)!?? i?,?, you?,,?,?,?,,? you, you,,, jerry: very that's) george: alright very see yeah he he well i'm) a is like go" no) in me no do- what that kramer: kramer: what me what with are of!?,?,,?,,,,? jerry: turns could know like george don't kramer: that kramer: me no) george: thought when know- of george: glasses george: alone was) in what kramer: me what that kramer: kramer: me me of!??,,, i,? i,?, you you you you??? i you,, you,?? jerry: water her have with are kramer: kramer: me no not on kramer: kramer: what kramer: what kramer

In [20]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 16
batch_size: 64
learning_rate: 0.0001
embedding_dim: 300
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: project glasses architect architect pretty alright could know he he don't for have that me kramer: no know don't kramer: kramer: no know he) in of!,,, you,? jerry: water her go" kramer: of! you?, jerry: would she your- kramer: george's was don't with kramer back are of! you? jerry: very he's come can't one at go they know- kramer:-!? you i jerry: what yeah- kramer: he my hey) a in of!? i?, i,,??,, jerry: when know he) george: water think do like go at all he) a a george: alright very gonna do don't that what that me what that no) a george: very see with was he) a george: thought very come one do don't with look one at have me kramer: kramer: me kramer: of! you i,, jerry: when know don't with look one do he don't yeah on me of a in of a a a a a!?,, i, jerry: would) a george: very he's gonna they know like george) in of!? i i?? jerry: thought one do- kramer: glasses talk thing if hey not-!,, you,, i,, you??, i,,, i,,?,?? i jerry: that kramer: what me! you you,,, you,,,,,?, you,,,?

In [19]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 8
batch_size: 256
learning_rate: 0.0001
embedding_dim: 1000
n_layers: 3


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: george's pretty her do he well don't yeah on me no) george: water could not-!,? jerry: water could do) a a is we know don't with kramer her out kramer if know- no) a a a a a is like have kramer: what that of in kramer:!?, jerry: would do he) a a! you,? you, i you you i you, i?,,,,,,, jerry: would)!,, i,? jerry: us thing uh) george: water think) george: would one they)!? you,,??,? jerry: turns was well he)!?,?, you i,, i you you i,? jerry: thought one do- kramer: thought could) a george: thought her have for! i,,? i,,? jerry: turns was)!, i you,???,,, jerry: would know- kramer:) in of george: alright one at all)! i, i jerry: so yeah like have that what of!?,,, jerry: went sorry could know he)!, i jerry: so for go they hey) george: don't for all)! jerry: of in kramer: what me of!, jerry: thought too think" no hey do he on kramer: of is don't me no do he he) george: thought too are that of! i i, jerry: alright see yeah- no not- of! i,,, jerry: very see with was)! i jerry: for have 

In [54]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 8
batch_size: 256
learning_rate: 0.0001
embedding_dim: 1000
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: project alright alright would one at george so that kramer: me me kramer: what kramer: me no do-!? you,, i? you you,, you,,,,,,, jerry: alright back where think- what that kramer: of! you i, i,, jerry: alright he's gonna do) george: so yeah-!? you??,, i you, i jerry: well) a! you?? i i you,, you jerry:!?, i, i?,? jerry: thought back was don't kramer: of!? you you?,, jerry: would that's hey know- no do- what that me of! you, you, you?,,,? you,, i?,? i?, i? you you you? i jerry: yeah he he) george: thought when that's) a! you,,,,? jerry: thought could) a george: thought could know on what that of a! jerry: kramer: kramer: no know like out look her all) a!? i?? jerry: water think know don't for go they hey hey)! you?, jerry: went very gonna good her all he) a!,,?, you??, you jerry: a a!,?? you you i?,,?, i, i,???,, i,, jerry: would) george: water her go" no hey not about- no do- kramer: have with kramer one" me me me no do he don't yeah he he i'm well i'm on of!?, you,,??, you, jer

In [51]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 8
batch_size: 256
learning_rate: 0.0001
embedding_dim: 300
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: george's alone alright thought very back kramer her out are kramer: of a a a a a a! i i,? i i,,,?, you,,, jerry: when do on of a!? i??? i you,, you, you, i, you,, i??? i? i,,,,,,?? jerry: alright her- no know like george) george: he) in of a is on of in kramer: kramer: what that kramer: what me kramer: kramer: what kramer: me me what me! jerry: kramer: me kramer: of a in of! i? jerry: turns are that no know about- what of a george: very see yeah like all don't with was don't that kramer: of a!??,,? you jerry:!?, jerry: very he's gonna your" me of in kramer: what with are of!? you i,, you i jerry: what kramer: no know- no not- no not like out look one at go" me what of a george: thought one do like go they know on what with was he he) george: water her all well well don't kramer: me me no) george: alright too could know like have that no hey know like out where was he well) in no not he well) in kramer: of in kramer:! jerry: kramer: kramer: of! you,? you? jerry: went her- kramer:

In [46]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 8
batch_size: 256
learning_rate: 0.0006
embedding_dim: 300
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: pretty thought alright very back are me kramer: of a george: alright could) a george: alright her- of a! jerry: of george: alright very come he's gonna your at have with was- of in what of a!, jerry: went back was) a is like all) in what of a a! you you, you,, you?? jerry: alright very one- of george: water are that kramer: me no know he)! you?????,?,, jerry: went back kramer if) george: thought her go they do- what kramer: me of!? you, jerry: thought when see for have kramer: what me kramer: of is we do on kramer: me what kramer: of!,?,, jerry: would she do don't for all) george:)! you jerry: a is don't kramer: me kramer: what that kramer: me me no not he on kramer:! i you, you you, you?,, jerry: very gonna your your at all well) george: alright one your do he on me kramer: of!??? i?, you i?,?,, you?, you?? you you you you?,, i,?,, jerry: would do- of!, you,,,, you, i?, you you, jerry: would know he so that me no know-! jerry: of in kramer:!? you, you you i,, jerry: very gonna 

In [41]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 8
batch_size: 256
learning_rate: 0.001
embedding_dim: 1000
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: george's alright alright sorry could not he don't for go do don't that of in what of george: water are with was- of a!,,, jerry: would do- what kramer: kramer: me of!, i,?, i, you, you,? i? i, you????, i?? i jerry: that kramer: me what that of in no do- what me of george: glasses george: alone talk thing uh do-! i,, jerry: very gonna" of a a a!, you? i,??,,?,,?,? you? you you,, you? i, jerry: would that's)! jerry: kramer: kramer: no not about-!,?, i i, you, jerry: would know he) george: thought too her have with kramer her have that what of! you jerry: george: alright very come see yeah he so with kramer one at have me of george: ready talk these one at out where if know like all) george: so that kramer: that kramer: what kramer: that of george: alright back are of george: ready was he well i'm don't that kramer: what with are that kramer: that kramer: of is- kramer: water her all well) george: water could not don't kramer: kramer: what of a a!,,, i i,,,?? i,, jerry: alright one

In [38]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 8
batch_size: 256
learning_rate: 0.001
embedding_dim: 300
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: pretty glasses pretty her do don't with kramer if know like all don't yeah- what that what that me kramer: no) george: alright when)!, i,?,, jerry: would that's do on kramer: kramer: kramer: no) a a in kramer: a!,,, i? i i you,,, you you,, jerry: alright when do- of george: very if hey) in me no not he he well my) george: very one" no not about" kramer: that of george: thought very see kramer: me of!,? you i you? jerry: alright when) a in!,,? you you i, you? you you jerry: what me!,, jerry: thought he's he's come back look one- kramer: glasses are kramer: kramer: me no)!?,, jerry: went sorry wait back was) george: thought very come one your" what with are kramer: what of a a a a is- kramer: water her all)!,?, jerry: would that's not- kramer: water are kramer: kramer: kramer: of a george: alright back where uh) in kramer: of george: water was he) a george: few thought back was well) george: alright too if) a!, i jerry: so kramer: that me of in kramer:!,? you??? i,,,,??,, jerry: v

In [35]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
print('sequence_length: {}\nbatch_size: {}\nlearning_rate: {}\nembedding_dim: {}\nn_layers: {}'.format(sequence_length,batch_size,learning_rate,embedding_dim,n_layers))

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

sequence_length: 8
batch_size: 256
learning_rate: 0.0001
embedding_dim: 300
n_layers: 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: project george's pretty alright very one at go your" me kramer: no) a in kramer:! jerry: what that no know he don't yeah he so with look one- no know on kramer: no hey do- kramer: alright one do- kramer: thought her go do like have with are that no do don't that me me what kramer: no do-!?, jerry: thought when do on what of a! you??,,?? i,,? jerry: us will one- of a george: very he's see with are with are for have kramer: of george: thought very see yeah like go" me no hey not he he well so kramer: of a in of george: alright her go do don't with was he he my do he he don't that kramer: that of george: ready was he don't kramer: that kramer: kramer: me no hey)!,, you,? jerry: thought back kramer think know like george don't that kramer: of!? you you you i,?,? you,, jerry: alright one do he he)!, you? jerry: thought back was) a is don't with are for george well on kramer: what that what with was) a a a!, i?? you you? i? jerry: turns are me of in kramer: what that of! you you,, you

In [19]:
"""
sequence_length: 8
batch_size: 128
learning_rate: 0.0001
embedding_dim: 1000
n_layers: 3
Training for 400 epoch(s)
"""
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: project glasses architect architect alone joke talk these her go your- kramer: water think" kramer: no do-! jerry: what kramer: no not- kramer: glasses alone talk thing if) a in kramer: what me of a! you, jerry: went sorry could hey) a a george: ready talk talk thing think do don't yeah like out look think do don't that kramer: of in what kramer: kramer: of!? jerry: by her have of a is he),,?,,, i,,,?,?? jerry: alright her- of george: ready talk talk thing uh) a a a a george: few she's her we not on no know on kramer: kramer: what of a a a george: ready are me no hey do he on me what me of a!? jerry: us talk thing think hey do),, you?? jerry: us us talk thing if do he), i jerry: well my do- kramer: water was)!,,,?? i?? jerry: turns her- of george: ready are of george: alright could know he he)!,?, i,? jerry: water could do) in of!,, you,,,, you jerry: george: very come he's come one your- what that no) george: would see for a a a!,, jerry: would do don't for have yeah don't for!

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [28]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.